In [2]:
# imports
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.preprocessing import image
from tensorflow.keras.layers import Input, Lambda ,Dense ,Flatten ,Dropout
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import cv2
import keras

In [3]:
# file paths
train_dir = "data/asl_alphabet_train"
eval_dir = "data/asl_alphabet_test"

In [4]:
# helper function to load training images
def load_training_images(directory):
    images = []
    labels = []
    for idx, label in enumerate(uniq_labels):
        for file in os.listdir(directory + "/" + label):
            filepath = directory + "/" + label + "/" + file
            # 200 x 200 is the image size
            image = cv2.resize(cv2.imread(filepath), (200, 200))
            images.append(image)
            labels.append(idx)
    images = np.array(images)
    labels = np.array(labels)
    return(images, labels)

In [5]:
uniq_labels = sorted(os.listdir(train_dir))
# remove hidden files
uniq_labels = [s for s in uniq_labels if s[0] != '.']
images, labels = load_training_images(directory = train_dir)


In [6]:
'''
# test data is formatted differently, so this isn't gonna work
if uniq_labels == sorted(os.listdir(eval_dir)):
    X_eval, y_eval = load_images(directory = eval_dir)
'''

"\n# test data is formatted differently, so this isn't gonna work\nif uniq_labels == sorted(os.listdir(eval_dir)):\n    X_eval, y_eval = load_images(directory = eval_dir)\n"

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2, stratify = labels)

n = len(uniq_labels)
train_n = len(X_train)
test_n = len(X_test)

print("Total number of symbols: ", n)
print("Number of training images: " , train_n)
print("Number of testing images: ", test_n)

'''
eval_n = len(X_eval)
print("Number of evaluation images: ", eval_n)
'''

Total number of symbols:  29
Number of training images:  69600
Number of testing images:  17400


'\neval_n = len(X_eval)\nprint("Number of evaluation images: ", eval_n)\n'

In [8]:
# one-hot encoding results
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)
#y_eval = keras.utils.to_categorical(y_eval)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(69600, 200, 200, 3) (69600, 29) (17400, 200, 200, 3) (17400, 29)


In [9]:
# normalizing images
X_train = X_train.astype('float32')/255.0
X_test = X_test.astype('float32')/255.0
#X_eval = X_eval.astype('float32')/255.0

In [10]:
#Initialising vgg16 
classifier_vgg16 = VGG16(input_shape= (200,200,3),include_top=False,weights='imagenet')

#don't train existing weights for vgg16
for layer in classifier_vgg16.layers:
    layer.trainable = False

In [11]:
classifier1 = classifier_vgg16.output #head mode
classifier1 = Flatten()(classifier1) #adding layer of flatten
classifier1 = Dense(units=256, activation='relu')(classifier1)
classifier1 = Dropout(0.6)(classifier1)
# units = number of classes = 29
classifier1 = Dense(units=29, activation='softmax')(classifier1)

# construct model
model = Model(inputs = classifier_vgg16.input , outputs = classifier1)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 50, 128)       0     

In [12]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq=1)
early = EarlyStopping(monitor='accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit(X_train,y_train,epochs=10,batch_size=64,validation_data= (X_test,y_test),callbacks=[checkpoint,early])
model.save('model_vgg16.h5')

Epoch 1/10

Epoch 00001: accuracy improved from -inf to 0.03125, saving model to vgg16_1.h5
   1/1088 [..............................] - ETA: 0s - loss: 3.8405 - accuracy: 0.0312
Epoch 00001: accuracy did not improve from 0.03125
   2/1088 [..............................] - ETA: 1:36:20 - loss: 4.7966 - accuracy: 0.0312
Epoch 00001: accuracy did not improve from 0.03125
   3/1088 [..............................] - ETA: 2:46:48 - loss: 5.0750 - accuracy: 0.0312
Epoch 00001: accuracy did not improve from 0.03125
   4/1088 [..............................] - ETA: 2:58:31 - loss: 4.9976 - accuracy: 0.0273
Epoch 00001: accuracy did not improve from 0.03125
   5/1088 [..............................] - ETA: 3:07:19 - loss: 4.7048 - accuracy: 0.0312
Epoch 00001: accuracy improved from 0.03125 to 0.03906, saving model to vgg16_1.h5
   6/1088 [..............................] - ETA: 3:14:52 - loss: 4.5005 - accuracy: 0.0391
Epoch 00001: accuracy improved from 0.03906 to 0.04018, saving model to vg

In [13]:
'''
# train model
history = model.fit(X_train, y_train, epochs = 10, batch_size = 64,validation_data=(X_test,y_test),)

# save model
model.save('model_vgg16.h5')
'''

"\n# train model\nhistory = model.fit(X_train, y_train, epochs = 10, batch_size = 64,validation_data=(X_test,y_test),)\n\n# save model\nmodel.save('model_vgg16.h5')\n"